In [134]:
%reload_ext autoreload
%autoreload 2

import os
import sys
import time
import cv2

from joblib import Parallel, delayed

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *

from matplotlib.path import Path
%matplotlib inline

from sklearn.cluster import KMeans
from sklearn.externals import joblib

import random
from collections import defaultdict

In [135]:
stack = 'MD589'
dm = DataManager(stack=stack)

first_detect_sec, last_detect_sec = detect_bbox_range_lookup[stack]

In [80]:
# Load sample locations

patches_rootdir = '/home/yuncong/CSHL_data_patches/'

import pandas as pd

table_filepath = os.path.join(patches_rootdir, '%(stack)s_indices_allLandmarks_allSections.h5'%{'stack':stack})
indices_allLandmarks_allSections = pd.read_hdf(table_filepath, 'indices_allLandmarks_allSections')
grid_parameters = pd.read_hdf(table_filepath, 'grid_parameters')

patch_size, stride, w, h = grid_parameters.tolist()
half_size = patch_size/2
ys, xs = np.meshgrid(np.arange(half_size, h-half_size, stride), np.arange(half_size, w-half_size, stride),
                 indexing='xy')
sample_locations = np.c_[xs.flat, ys.flat]

Opening /home/yuncong/CSHL_data_patches/MD589_indices_allLandmarks_allSections.h5 in read-only mode
Opening /home/yuncong/CSHL_data_patches/MD589_indices_allLandmarks_allSections.h5 in read-only mode


In [81]:
# print patch_size, stride, w, h
# print np.arange(half_size, h-half_size, stride)
# print sample_locations

224 56 13408 11744


In [82]:
labels = ['BackG', '5N', '7n', '7N', '12N', 'Pn', 'VLL', 
          '6N', 'Amb', 'R', 'Tz', 'RtTg', 'LRt', 'LC', 'AP', 'sp5']

n_labels = len(labels)

labels_index = dict((j, i) for i, j in enumerate(labels))

labels_from_surround = dict( (l+'_surround', l) for l in labels[1:])

labels_surroundIncluded_list = labels[1:] + [l+'_surround' for l in labels[1:]]
labels_surroundIncluded = set(labels_surroundIncluded_list)

labels_surroundIncluded_index = dict((j, i) for i, j in enumerate(labels_surroundIncluded_list))

# colors = np.random.randint(0, 255, (len(labels_index), 3))
colors = np.loadtxt(os.environ['REPO_DIR'] + '/visualization/100colors.txt')
colors[labels_index['BackG']] = 1.

In [83]:
xmin, ymin, w, h = detect_bbox_lookup[stack]
xmin = xmin * 32
ymin = ymin * 32
w = w * 32
h = h * 32
xmax = xmin + w - 1
ymax = ymin + h - 1

In [84]:
output_dir = '/oasis/projects/nsf/csd395/yuncong/Brain/learning/sift'
M = 200 # vocabulary size
colors = np.vstack([(0,0,0), np.random.randint(0, 255, (M, 3))])

In [85]:
# Load vocabulary (as a sklearn.KMeans object)

if os.path.exists(output_dir + '/vocab.pkl'):
    
    # Load vocabulary
    vocabulary = joblib.load(output_dir + '/vocab.pkl')
    
# else:
    
#     if os.path.exists(output_dir + '/sift_descriptors_pool_arr.bp'):
        
#         # Load descriptor pool
#         descriptors_pool_arr = bp.unpack_ndarray_file(output_dir + '/sift_descriptors_pool_arr.bp')

#         t = time.time()

#         vocabulary = KMeans(init='random', n_clusters=M, n_init=10)
#         vocabulary.fit(descriptors_pool_arr)

#         sys.stderr.write('sift: %.2f seconds\n' % (time.time() - t)) # 300 seconds

#         cluster_centers = vocabulary.cluster_centers_

#         joblib.dump(vocabulary, output_dir + '/vocab.pkl')

#     else:
        
#         # Generate SIFT descriptor pool
#         descriptors_pool = []

#         sift = cv2.SIFT();
        
#         for sec in range(first_detect_sec, last_detect_sec+1, 10):

#             print sec

#             dm.set_slice(sec)
#             dm._load_image(versions=['rgb-jpg'])
#             img = dm.image_rgb_jpg[ymin:ymax+1, xmin:xmax+1]

#             keypoints, descriptors = sift.detectAndCompute(img, None)

#             n = 1000
#             random_indices = np.random.choice(range(len(descriptors)), n, replace=False)

#             descriptors_pool.append(descriptors[random_indices])

#         descriptors_pool_arr = np.vstack(descriptors_pool)
#         print len(descriptors_pool_arr), 'in descriptor pool'

#         bp.pack_ndarray_file(descriptors_pool_arr, output_dir + '/sift_descriptors_pool_arr.bp')

In [87]:
# print first_detect_sec, last_detect_sec
# print range(first_detect_sec, last_detect_sec+1, 10), len(range(first_detect_sec, last_detect_sec+1, 10))

150 316
[150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250, 260, 270, 280, 290, 300, 310] 17


In [101]:
# l = 2

# grid_size = patch_size / 2**l

# if l == 2:
#     rx = [-2, -1, 0, 1]
#     ry = [-2, -1, 0, 1]
# elif l == 1:
#     rx = [-1, 0]
#     ry = [-1, 0]
# elif l == 0:
#     rx = [-.5]
#     ry = [-.5]

# rxs, rys = np.meshgrid(rx, ry)
# print 'Grid size: ', grid_size

# patch_coords_allGrid = []

# for grid_i, (rx, ry) in enumerate(np.c_[rxs.flat, rys.flat]):
#     print rx*grid_size, ry*grid_size, (rx+1)*grid_size, (ry+1)*grid_size

#     patch_xmin = sample_locs_roi[:,0] + rx * grid_size
#     patch_ymin = sample_locs_roi[:,1] + ry * grid_size
#     patch_xmax = sample_locs_roi[:,0] + (rx + 1) * grid_size
#     patch_ymax = sample_locs_roi[:,1] + (ry + 1) * grid_size

#     patch_coords_allGrid.append([patch_xmin, patch_ymin, patch_xmax, patch_ymax])
#     print (patch_xmin[0], patch_ymin[0], patch_xmax[0], patch_ymax[0])


In [48]:
sift = cv2.SIFT();

first_detect_sec, last_detect_sec = detect_bbox_range_lookup[stack]

progress_bar = FloatProgress(min=first_detect_sec, max=last_detect_sec)
display(progress_bar)

hists0_allLandmarks = defaultdict(list)
hists1_allLandmarks = defaultdict(list)
hists2_allLandmarks = defaultdict(list)

fname_allLandmarks = defaultdict(list)
total_t = time.time()

for sec in range(first_detect_sec, last_detect_sec+1):    
# for sec in [first_detect_sec, first_detect_sec+1]:
# for sec in [first_detect_sec]:
    
    progress_bar.value = sec
    
    labelmap_fp = output_dir + '/%(stack)s/%(stack)s_%(sec)04d_labelmap.hdf' % {'stack': stack, 'sec': sec}
    
    if os.path.exists(labelmap_fp):
        
        # Load labelmap
        labelmap = load_hdf(labelmap_fp)
        
#     else:
        
#         # Compute keypoints and assign labels

#         dm.set_slice(sec)
#         dm._load_image(versions=['rgb-jpg'])

#         img = dm.image_rgb_jpg[ymin:ymax+1, xmin:xmax+1]

#         t = time.time()
#         keypoints, descriptors = sift.detectAndCompute(img, None); # 128 dim descriptor ～ 120 seconds
#         sys.stderr.write('sift: %.2f seconds\n' % (time.time() - t)) 

#         keypoints_arr = np.array([k.pt for k in keypoints])
#         print len(keypoints), 'keypoints' # ~ 500k

#         t = time.time()
#         keypoint_labels = vocabulary.predict(descriptors)
#         sys.stderr.write('predict: %.2f seconds\n' % (time.time() - t))  # ~ 20 s

#         # visualize keypoints (color indicating label)

#         # viz = img.copy()
#         # for (x, y), l in zip(keypoints_arr, keypoint_labels):
#         #     cv2.circle(viz, (int(x), int(y)), 3, colors[l], -1)
#         # display_image(viz)

#         # generate labelmap

#         labelmap = np.zeros(dm.image_rgb_jpg.shape[:2], np.int)
#         keypoints_arr_int = np.floor(keypoints_arr + (xmin, ymin)).astype(np.int)  # coords on original image
#         labelmap[keypoints_arr_int[:,1], keypoints_arr_int[:,0]] = keypoint_labels + 1

#         save_hdf(labelmap, labelmap_fp)
        

    # Compute histograms (method 2), for all levels

    num_sample = 30
    
    for name in labels[1:]:
        
        if name not in indices_allLandmarks_allSections[sec].dropna().index: 
            continue

        indices = indices_allLandmarks_allSections[sec][name]
        indices_sampled = np.random.choice(indices, min(len(indices), num_sample), replace=False)
    
        sample_locs_roi = sample_locations[indices_sampled]
        
        # MY ADDITION
        # compute level-0 jpeg
        l = 0
        grid_size = patch_size / 2**l
        rx = [-.5]
        ry = [-.5]
        rxs, rys = np.meshgrid(rx, ry)
        patch_coords_allGrid = []
        for grid_i, (rx, ry) in enumerate(np.c_[rxs.flat, rys.flat]):
            patch_xmin = sample_locs_roi[:,0] + rx * grid_size
            patch_ymin = sample_locs_roi[:,1] + ry * grid_size
            patch_xmax = sample_locs_roi[:,0] + (rx + 1) * grid_size
            patch_ymax = sample_locs_roi[:,1] + (ry + 1) * grid_size
            patch_coords_allGrid.append([patch_xmin, patch_ymin, patch_xmax, patch_ymax])
        all_coords = np.hstack(patch_coords_allGrid)
        patch_xmin = all_coords[0]
        patch_ymin = all_coords[1]
        patch_xmax = all_coords[2]
        patch_ymax = all_coords[3]
        mydir = '/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg/train'
        dm.set_slice(sec)
        dm._load_image(versions=['rgb-jpg'])
        fnames = np.zeros((len(patch_xmin),1)).astype('|S255')
        for p in xrange(len(patch_xmin)):
            fname = mydir+'/%(stack)s/%(stack)s_%(name)s_%(sec)04d_%(patch)04d.png'% \
                            {'stack': stack, 'name': name, 'sec': sec, 'patch': p}
            fnames[p] = fname
            img = dm.image_rgb_jpg[patch_ymin[p]:patch_ymax[p]+1, patch_xmin[p]:patch_xmax[p]+1]
            plt.imsave(fname, img)
            # plt.imshow(img, cmap=plt.cm.gray)
        fname_allLandmarks[name].append(fnames)
        print ''
        print (sec, name)
        print 'Size of patches: (%f by %f)' % (abs(patch_xmax[0]-patch_xmin[0]), abs(patch_ymax[0]-patch_ymin[0]))
        print 'Num of patches: ', len(patch_xmin)
        print 'Num of fnames: ', len(fname_allLandmarks[name])
        # MY ADDITION END

        # compute level-2 histograms
        l = 2

        grid_size = patch_size / 2**l

        if l == 2:
            rx = [-2, -1, 0, 1]
            ry = [-2, -1, 0, 1]
        elif l == 1:
            rx = [-1, 0]
            ry = [-1, 0]
        elif l == 0:
            rx = [-.5]
            ry = [-.5]

        rxs, rys = np.meshgrid(rx, ry)

        patch_coords_allGrid = []

        for grid_i, (rx, ry) in enumerate(np.c_[rxs.flat, rys.flat]):

            patch_xmin = sample_locs_roi[:,0] + rx * grid_size
            patch_ymin = sample_locs_roi[:,1] + ry * grid_size
            patch_xmax = sample_locs_roi[:,0] + (rx + 1) * grid_size
            patch_ymax = sample_locs_roi[:,1] + (ry + 1) * grid_size

            patch_coords_allGrid.append([patch_xmin, patch_ymin, patch_xmax, patch_ymax])


        all_coords = np.hstack(patch_coords_allGrid)
        patch_xmin = all_coords[0]
        patch_ymin = all_coords[1]
        patch_xmax = all_coords[2]
        patch_ymax = all_coords[3]

        def compute_histogram_particular_label(i):
            m = (labelmap == i).astype(np.uint8)
            mi = cv2.integral(m)
            ci = mi[patch_ymin, patch_xmin] + mi[patch_ymax, patch_xmax] - mi[patch_ymax, patch_xmin] - mi[patch_ymin, patch_xmax]
            return ci

        t = time.time()
        hists = Parallel(n_jobs=16)(delayed(compute_histogram_particular_label)(i) for i in range(1, M+1))
        sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # ~ 18 seconds

        n_grid = (2**l)**2
        hists_arr2 = np.transpose(np.reshape(hists, (M, n_grid, -1)))
#         print hists_arr2.shape

        # compute level-1 histograms based on level-2 histograms

        hists_arr1 = np.transpose([hists_arr2[:, [0,1,4,5], :].sum(axis=1),
                                   hists_arr2[:, [2,3,6,7], :].sum(axis=1),
                                   hists_arr2[:, [8,9,12,13], :].sum(axis=1),
                                   hists_arr2[:, [10,11,14,15], :].sum(axis=1)], 
                                  [1,0,2])
#         print hists_arr1.shape

        # compute level-0 histograms based on level-1 histograms

        hists_arr0 = hists_arr1.sum(axis=1)
#         print hists_arr0.shape

        hists2_allLandmarks[name].append(hists_arr2)
        hists1_allLandmarks[name].append(hists_arr1)
        hists0_allLandmarks[name].append(hists_arr0)
        
#     bp.pack_ndarray_file(hists_arr0, output_dir + '/%(stack)s_%(sec)04d_roi1_histograms_l0.bp' % {'stack': stack, 'sec': sec})
#     bp.pack_ndarray_file(hists_arr1, output_dir + '/%(stack)s_%(sec)04d_roi1_histograms_l1.bp' % {'stack': stack, 'sec': sec})
#     bp.pack_ndarray_file(hists_arr2, output_dir + '/%(stack)s_%(sec)04d_roi1_histograms_l2.bp' % {'stack': stack, 'sec': sec})

    # save_hdf(hists_arr0, output_dir + '/%(stack)s_%(sec)04d_roi1_histograms_l0.hdf' % {'stack': stack, 'sec': sec})
    # save_hdf(hists_arr1, output_dir + '/%(stack)s_%(sec)04d_roi1_histograms_l1.hdf' % {'stack': stack, 'sec': sec})
    # save_hdf(hists_arr2, output_dir + '/%(stack)s_%(sec)04d_roi1_histograms_l2.hdf' % {'stack': stack, 'sec': sec})
    
hists0_allLandmarks.default_factory = None
hists1_allLandmarks.default_factory = None
hists2_allLandmarks.default_factory = None

fname_allLandmarks.default_factory = None
sys.stderr.write('Total doone in %f seconds\n' % (time.time() - total_t))
    


(150, '5N')
Size of patches: (224.000000 by 224.000000)
Num of patches:  30
Num of fnames:  1


/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/ipykernel/__main__.py:104: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future



(150, '7n')
Size of patches: (224.000000 by 224.000000)
Num of patches:  30
Num of fnames:  1


done in 12.733136 seconds



(150, 'VLL')
Size of patches: (224.000000 by 224.000000)
Num of patches:  30
Num of fnames:  1


...

done in 13.130759 seconds



(316, '7N')
Size of patches: (224.000000 by 224.000000)
Num of patches:  30
Num of fnames:  86


done in 12.915774 seconds



(316, 'sp5')
Size of patches: (224.000000 by 224.000000)
Num of patches:  30
Num of fnames:  108


done in 14.043834 seconds
done in 13.547384 seconds
Total doone in 17080.235514 seconds


In [49]:
print hists0_allLandmarks.keys()
print fname_allLandmarks.keys()

['RtTg', 'Tz', 'VLL', '12N', '7n', 'Amb', 'sp5', '6N', 'AP', '5N', 'LRt', '7N', 'R', 'Pn', 'LC']
['RtTg', 'Tz', 'VLL', '12N', '7n', 'Amb', 'sp5', '6N', 'AP', '5N', 'LRt', '7N', 'R', 'Pn', 'LC']


In [50]:
fname_allLandmarks = {name: np.concatenate(arrs) for name, arrs in fname_allLandmarks.iteritems()}
hists0_allLandmarks = {name: np.concatenate(arrs) for name, arrs in hists0_allLandmarks.iteritems()}
hists1_allLandmarks = {name: np.concatenate(arrs) for name, arrs in hists1_allLandmarks.iteritems()}
hists2_allLandmarks = {name: np.concatenate(arrs) for name, arrs in hists2_allLandmarks.iteritems()}

In [91]:
print hists0_allLandmarks.keys()
print fname_allLandmarks.keys()

['Tz', 'VLL', '7n', 'Amb', 'sp5', '12N', 'AP', '6N', '5N', 'RtTg', '7N', 'R', 'LC', 'Pn', 'LRt']
['Tz', 'VLL', '7n', 'Amb', 'sp5', '12N', 'AP', '6N', '5N', 'RtTg', '7N', 'R', 'LC', 'Pn', 'LRt']


In [89]:
print fname_allLandmarks['Tz']

[[ '/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg/train/MD589/MD589_Tz_0197_0000.png']
 [ '/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg/train/MD589/MD589_Tz_0197_0001.png']
 [ '/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg/train/MD589/MD589_Tz_0197_0002.png']
 ..., 
 [ '/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg/train/MD589/MD589_Tz_0276_0006.png']
 [ '/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg/train/MD589/MD589_Tz_0276_0007.png']
 [ '/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg/train/MD589/MD589_Tz_0276_0008.png']]


In [51]:
for name, arr in fname_allLandmarks.iteritems():
    print arr.shape, arr.dtype
for name, arr in hists0_allLandmarks.iteritems():
    print name, arr.shape
for name, arr in hists1_allLandmarks.iteritems():
    print name, arr.shape
for name, arr in hists2_allLandmarks.iteritems():
    print name, arr.shape

(1387, 1) |S255
(1287, 1) |S255
(3444, 1) |S255
(346, 1) |S255
(3240, 1) |S255
(1230, 1) |S255
(483, 1) |S255
(154, 1) |S255
(1440, 1) |S255
(2639, 1) |S255
(2579, 1) |S255
(1082, 1) |S255
(481, 1) |S255
(3042, 1) |S255
(1050, 1) |S255
Tz (1387, 200)
VLL (1287, 200)
7n (3444, 200)
Amb (346, 200)
sp5 (3240, 200)
12N (1230, 200)
AP (483, 200)
6N (154, 200)
5N (1440, 200)
RtTg (2639, 200)
7N (2579, 200)
R (1082, 200)
LC (481, 200)
Pn (3042, 200)
LRt (1050, 200)
Tz (1387, 4, 200)
VLL (1287, 4, 200)
7n (3444, 4, 200)
Amb (346, 4, 200)
sp5 (3240, 4, 200)
12N (1230, 4, 200)
AP (483, 4, 200)
6N (154, 4, 200)
5N (1440, 4, 200)
RtTg (2639, 4, 200)
7N (2579, 4, 200)
R (1082, 4, 200)
LC (481, 4, 200)
Pn (3042, 4, 200)
LRt (1050, 4, 200)
Tz (1387, 16, 200)
VLL (1287, 16, 200)
7n (3444, 16, 200)
Amb (346, 16, 200)
sp5 (3240, 16, 200)
12N (1230, 16, 200)
AP (483, 16, 200)
6N (154, 16, 200)
5N (1440, 16, 200)
RtTg (2639, 16, 200)
7N (2579, 16, 200)
R (1082, 16, 200)
LC (481, 16, 200)
Pn (3042, 16, 200

In [136]:
output_dir = '/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg/'

In [52]:
t = time.time()

for name, arr in hists0_allLandmarks.iteritems():
    bp.pack_ndarray_file(arr, output_dir + '/train/%(stack)s_%(name)s_histograms_l0.bp' % {'stack': stack, 'name': name})

for name, arr in hists1_allLandmarks.iteritems():
    bp.pack_ndarray_file(arr, output_dir + '/train/%(stack)s_%(name)s_histograms_l1.bp' % {'stack': stack, 'name': name})

for name, arr in hists2_allLandmarks.iteritems():
    bp.pack_ndarray_file(arr, output_dir + '/train/%(stack)s_%(name)s_histograms_l2.bp' % {'stack': stack, 'name': name})
    
for name, arr in fname_allLandmarks.iteritems():
    bp.pack_ndarray_file(arr, output_dir + '/train/%(stack)s_%(name)s_fnames.bp' % {'stack': stack, 'name': name})

sys.stderr.write('Done in %f seconds\n' % (time.time() - t)) # ~ 18 seconds

Done in 0.908770 seconds


In [ ]:
sift = cv2.SIFT();

first_detect_sec, last_detect_sec = detect_bbox_range_lookup[stack]

progress_bar = FloatProgress(min=first_detect_sec, max=last_detect_sec)
display(progress_bar)

total_t = time.time()

F = []
H0 = []
H1 = []
H2 = []

# for sec in range(first_detect_sec, last_detect_sec+1):
# for sec in range(first_detect_sec, first_detect_sec+2):
for sec in range(first_detect_sec, last_detect_sec+1):    
    
    progress_bar.value = sec
    
    labelmap_fp = output_dir + '/%(stack)s/%(stack)s_%(sec)04d_labelmap.hdf' % {'stack': stack, 'sec': sec}
    
    if os.path.exists(labelmap_fp):
        
        # Load labelmap
        labelmap = load_hdf(labelmap_fp)
 
    # Compute histograms (method 2), for all levels

    num_sample = 5
    patch_count = 0
    
    for name in labels_surround:
        
        if name not in indices_allLandmarks_allSections[sec].dropna().index: 
            continue

        indices = indices_allLandmarks_allSections[sec][name]
        indices_sampled = np.random.choice(indices, min(len(indices), num_sample), replace=False)
    
        sample_locs_roi = sample_locations[indices_sampled]
        
        # MY ADDITION
        # compute level-0 jpeg
        l = 0
        grid_size = patch_size / 2**l
        rx = [-.5]
        ry = [-.5]
        rxs, rys = np.meshgrid(rx, ry)
        patch_coords_allGrid = []
        for grid_i, (rx, ry) in enumerate(np.c_[rxs.flat, rys.flat]):
            patch_xmin = sample_locs_roi[:,0] + rx * grid_size
            patch_ymin = sample_locs_roi[:,1] + ry * grid_size
            patch_xmax = sample_locs_roi[:,0] + (rx + 1) * grid_size
            patch_ymax = sample_locs_roi[:,1] + (ry + 1) * grid_size
            patch_coords_allGrid.append([patch_xmin, patch_ymin, patch_xmax, patch_ymax])
        all_coords = np.hstack(patch_coords_allGrid)
        patch_xmin = all_coords[0]
        patch_ymin = all_coords[1]
        patch_xmax = all_coords[2]
        patch_ymax = all_coords[3]
        mydir = '/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg/train'
        dm.set_slice(sec)
        dm._load_image(versions=['rgb-jpg'])
        fnames = np.zeros((len(patch_xmin),1)).astype('|S255')
        for p in xrange(len(patch_xmin)):
            fname = mydir+'/%(stack)s/%(stack)s_%(name)s_%(sec)04d_%(patch)04d.png'% \
                            {'stack': stack, 'name': 'BackG', 'sec': sec, 'patch': patch_count}
            fnames[p] = fname
            patch_count += 1
            img = dm.image_rgb_jpg[patch_ymin[p]:patch_ymax[p]+1, patch_xmin[p]:patch_xmax[p]+1]
            plt.imsave(fname, img)
            # plt.imshow(img, cmap=plt.cm.gray)
            
        F.append(fnames)
        print ''
        print (sec, name)
        print 'Size of patches: (%f by %f)' % (abs(patch_xmax[0]-patch_xmin[0]), abs(patch_ymax[0]-patch_ymin[0]))
        print 'Num of patches: ', len(patch_xmin)
        print 'Num of fnames: ', len(F)
        # MY ADDITION END

        # compute level-2 histograms
        l = 2

        grid_size = patch_size / 2**l

        if l == 2:
            rx = [-2, -1, 0, 1]
            ry = [-2, -1, 0, 1]
        elif l == 1:
            rx = [-1, 0]
            ry = [-1, 0]
        elif l == 0:
            rx = [-.5]
            ry = [-.5]

        rxs, rys = np.meshgrid(rx, ry)

        patch_coords_allGrid = []

        for grid_i, (rx, ry) in enumerate(np.c_[rxs.flat, rys.flat]):

            patch_xmin = sample_locs_roi[:,0] + rx * grid_size
            patch_ymin = sample_locs_roi[:,1] + ry * grid_size
            patch_xmax = sample_locs_roi[:,0] + (rx + 1) * grid_size
            patch_ymax = sample_locs_roi[:,1] + (ry + 1) * grid_size

            patch_coords_allGrid.append([patch_xmin, patch_ymin, patch_xmax, patch_ymax])


        all_coords = np.hstack(patch_coords_allGrid)
        patch_xmin = all_coords[0]
        patch_ymin = all_coords[1]
        patch_xmax = all_coords[2]
        patch_ymax = all_coords[3]

        def compute_histogram_particular_label(i):
            m = (labelmap == i).astype(np.uint8)
            mi = cv2.integral(m)
            ci = mi[patch_ymin, patch_xmin] + mi[patch_ymax, patch_xmax] - mi[patch_ymax, patch_xmin] - mi[patch_ymin, patch_xmax]
            return ci

        t = time.time()
        hists = Parallel(n_jobs=16)(delayed(compute_histogram_particular_label)(i) for i in range(1, M+1))
        sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # ~ 18 seconds

        n_grid = (2**l)**2
        hists_arr2 = np.transpose(np.reshape(hists, (M, n_grid, -1)))
#         print hists_arr2.shape

        # compute level-1 histograms based on level-2 histograms

        hists_arr1 = np.transpose([hists_arr2[:, [0,1,4,5], :].sum(axis=1),
                                   hists_arr2[:, [2,3,6,7], :].sum(axis=1),
                                   hists_arr2[:, [8,9,12,13], :].sum(axis=1),
                                   hists_arr2[:, [10,11,14,15], :].sum(axis=1)], 
                                  [1,0,2])
#         print hists_arr1.shape

        # compute level-0 histograms based on level-1 histograms

        hists_arr0 = hists_arr1.sum(axis=1)
#         print hists_arr0.shape

        H2.append(hists_arr2)
        H1.append(hists_arr1)
        H0.append(hists_arr0)


sys.stderr.write('Total doone in %f seconds\n' % (time.time() - total_t)) # ~ 18 seconds


In [143]:
labels_surround = labels_from_surround.keys()
print labels_surround

Setting environment for Gordon
Setting environment for Gordon

[autoreload of utilities2015 failed: Traceback (most recent call last):
  File "/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
AttributeError: __abstractmethods__
]
[autoreload of utilities2015 failed: Traceback (most recent call last):
  File "/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
AttributeError: __abstractmethods__
]


Setting environment for Gordon


[autoreload of utilities2015 failed: Traceback (most recent call last):
  File "/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
AttributeError: __abstractmethods__
]


Setting environment for Gordon

...


Setting environment for Gordon


[autoreload of utilities2015 failed: Traceback (most recent call last):
  File "/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
AttributeError: __abstractmethods__
]


Setting environment for Gordon


[autoreload of utilities2015 failed: Traceback (most recent call last):
  File "/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
AttributeError: __abstractmethods__
]


['7n_surround', 'LRt_surround', '7N_surround', '6N_surround', 'sp5_surround', 'Tz_surround', '5N_surround', '12N_surround', 'AP_surround', 'Amb_surround', 'VLL_surround', 'Pn_surround', 'RtTg_surround', 'LC_surround', 'R_surround']


In [144]:
F = np.concatenate(F)
H0 = np.concatenate(H0)
H1 = np.concatenate(H1)
H2 = np.concatenate(H2)

In [145]:
print F[0]
print F.shape, F.dtype
print H0.shape
print H1.shape
print H2.shape

/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg/train/MD589/MD589_BackG_0150_0000.png
(5105,) |S255
(1021000,)
(20420, 200)
(81680, 200)


In [150]:
# F = F.reshape((5105,1))
# H0 = H0.reshape((5105,200))
# H1 = H1.reshape((5105,4,200))
# H2 = H2.reshape((5105,16,200))

In [151]:
# print F.shape, F.dtype
# print H0.shape
# print H1.shape
# print H2.shape

(5105, 1) |S255
(5105, 200)
(5105, 4, 200)
(5105, 16, 200)


In [152]:
output_dir + '/train/%(stack)s_%(name)s_histograms_l0.bp' % {'stack': stack, 'name': 'BackG'}

'/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg//train/MD589_BackG_histograms_l0.bp'

In [154]:
bp.pack_ndarray_file(H0, output_dir + '/train/%(stack)s_%(name)s_histograms_l0.bp' % {'stack': stack, 'name': 'BackG'})
bp.pack_ndarray_file(H1, output_dir + '/train/%(stack)s_%(name)s_histograms_l1.bp' % {'stack': stack, 'name': 'BackG'})
bp.pack_ndarray_file(H2, output_dir + '/train/%(stack)s_%(name)s_histograms_l2.bp' % {'stack': stack, 'name': 'BackG'})
bp.pack_ndarray_file(F, output_dir + '/train/%(stack)s_%(name)s_fnames.bp' % {'stack': stack, 'name': 'BackG'}, chunk_size=255*4000)